In [2]:
# Prerequisets
!pip install pynetlogo
!pip install JPype1

     |████████████████████████████████| 48 kB 182 kB/s 
  Created wheel for pynetlogo: filename=pyNetLogo-0.4.1-py3-none-any.whl size=64516 sha256=bf8a40c5e8446cbca9adf286f1779b182bc7476ef5778e9660547ac4000fd1dc
  Stored in directory: /Users/ricoherzog/Library/Caches/pip/wheels/73/a0/47/93a2adf66bc7381ec21a1be6be2f774a49e4a5ee7a9ef21e54
Successfully built pynetlogo


In [1]:
# import all necessary python libraries and the pyNetLogo library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
import pyNetLogo

from SALib.sample import latin
from SALib.analyze import sobol

In [2]:
# set up the model
netlogo = pyNetLogo.NetLogoLink(gui=False) #Mac users should set this to False
netlogo.load_model('Model.nlogo')

# Setup Experiments

In [65]:
# Parameters
no_samples = 5
no_repetitions = 2

In [66]:
problem = {
  'num_vars': 7,
  'names': ['administrative-network-meetings',
           'total-project-proposal-frequency',
           'informal-meetings-frequency',
           'green-energy-openness-change',
           'political-variety-change',
           'max-project-capacity',
           'random-intial-trust'],
  'bounds': [[0, 25],
            [0, 25],
            [0, 25],
            [-5,5],
            [-5,5],
            [0, 25],
            [0, 1]]
}

param_values = np.round(latin.sample(problem, no_samples))
df = pd.DataFrame(param_values)
df.iloc[:,-1] = df.iloc[:,-1].astype("bool")
param_values = np.array(df)
param_values

array([[15.0, 21.0, 2.0, -2.0, 4.0, 7.0, False],
       [1.0, 18.0, 21.0, 4.0, 1.0, 21.0, False],
       [18.0, 4.0, 19.0, -0.0, -1.0, 18.0, True],
       [7.0, 13.0, 12.0, -4.0, -2.0, 15.0, True],
       [24.0, 8.0, 8.0, 1.0, -3.0, 1.0, False]], dtype=object)

In [68]:
idx = pd.MultiIndex.from_product([[],[]],names=['Run', 'Random Seed'])
col = ['MW_implemented_wind', 'MW_implemented_solar', 'MW_implemented_urban']
results = pd.DataFrame('', idx, col)

for run in range(param_values.shape[0]):
    for rs in range(no_repetitions):
        netlogo.command('setup')
        netlogo.command('random-seed {}'.format(rs))
        
        # Set the input parameters
        for i, name in enumerate(problem['names']):
            # Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, param_values[run,i]))


        # Run for 100 ticks and return the number of sheep and wolf agents at 
        # each time step
        MW_implemented = netlogo.repeat_report(['current-wind-production','current-solar-production', 'current-urban-production'], 360)
    
        # For each run, save the value of MW_implemented
        results.loc[(run, rs), 'MW_implemented_wind'] = MW_implemented['current-wind-production'].max()
        results.loc[(run, rs), 'MW_implemented_solar'] = MW_implemented['current-solar-production'].max()
        results.loc[(run, rs), 'MW_implemented_urban'] = MW_implemented['current-urban-production'].max()

In [69]:
results

MW_implemented_wind MW_implemented_solar MW_implemented_urban
Run Random Seed                                                              
0   0                          55.0                238.0                103.5
    1                          30.0                149.0                123.0
1   0                           5.0                 60.0                 99.0
    1                          15.0                 15.0                 76.5
2   0                           0.0                 60.0                 21.0
    1                           0.0                  0.0                 13.5
3   0                           5.0                 60.0                 66.0
    1                          10.0                 75.0                 63.0
4   0                           5.0                  0.0                 34.5
    1                          10.0                 15.0                 40.5

In [ ]:
#Set the input parameters
for i, name in enumerate(problem['names']):
    if name == 'random-seed':
        #The NetLogo random seed requires a different syntax
        netlogo.command('random-seed {}'.format(i))
    else:
        #Otherwise, assume the input parameters are global variables
        netlogo.command('set {0} {1}'.format(name, experiment[i]))

netlogo.command('setup')
#Run for 100 ticks and return the number of sheep and wolf agents at each time step
MW_implemented = netlogo.repeat_report(['current-renewable-production "wind"','current-renewable-production "solar"', 'current-renewable-production "urban"'], 100)

In [19]:
# set up a simulation function that takes a single experiment (i.e. a vector of input parameters)
# as an argument, and returns the outcomes of interest in a pandas Series

def simulation(experiment):

    #Set the input parameters
    for i, name in enumerate(problem['names']):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(experiment[i]))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, experiment[i]))

    netlogo.command('setup')
    #Run for 100 ticks and return the number of sheep and wolf agents at each time step
    MW_implemented = netlogo.repeat_report(['current-renewable-production "wind"','current-renewable-production "solar"', 'current-renewable-production "urban"'], 100)


    return MW_implemented

In [18]:
# to close the netlogo workspace
netlogo.kill_workspace()